Framewise

In [1]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import glob
import seaborn as sns
import matplotlib.pyplot as plt

def load_video_frames(video_path, num_frames=8):
    """
    从单个视频文件中读取帧
    """
    frames = []
    try:
        container = av.open(video_path)
        for frame in container.decode(video=0):
            frames.append(frame.to_rgb().to_ndarray())
            if len(frames) >= num_frames:
                return np.stack(frames[:num_frames])
    except Exception as e:
        print(f"Error processing video {video_path}: {str(e)}")
        raise
    
    if len(frames) == 0:
        raise ValueError("No frames could be extracted")
    
    # 如果帧数不足，重复最后一帧
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames[:num_frames])

def load_labels(label_file):
    """
    加载标签文件
    """
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def plot_confusion_matrix(true_labels, predictions, save_path):
    """
    绘制并保存混淆矩阵
    """
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path)
    plt.close()

def evaluate_videos(config):
    # 设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 初始化模型
    print("Loading model...")
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    model.eval()  # 设置为评估模式
    
    # 加载标签
    print("Loading labels...")
    video_labels = load_labels(config['label_file'])
    
    # 准备结果存储
    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'confidence_scores': [],
        'model_name': config['model_name'],
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config
    }
    
    # 获取所有视频文件
    video_files = glob.glob(os.path.join(config['video_dir'], '*.mp4'))
    print(f"Found {len(video_files)} video files")
    
    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42))
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
            print(f"Sampled {config['sample_size']} videos for evaluation")
    
    # 处理每个视频
    error_count = 0
    for video_path in tqdm(video_files, desc="Processing videos"):
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # 跳过没有标签的视频
        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        
        try:
            # 读取视频帧并进行预测
            frames = load_video_frames(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probabilities = torch.nn.functional.softmax(logits, dim=-1)
                pred_label = logits.argmax(-1).cpu().numpy()[0]
                confidence = float(probabilities.max().cpu().numpy())
            
            # 存储结果
            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            detailed_results['confidence_scores'].append(confidence)
            
        except Exception as e:
            print(f"Error processing video {video_name}: {str(e)}")
            error_count += 1
            continue
    
    print(f"\nProcessing completed with {error_count} errors")
    
    # 计算评价指标
    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    
    # 添加汇总指标
    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions']),
        'error_count': error_count,
        'average_confidence': float(np.mean(detailed_results['confidence_scores']))
    }
    
    # 打印结果
    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Average Confidence: {detailed_results['summary']['average_confidence']:.4f}")
    
    # 保存结果
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = detailed_results['timestamp']
    base_name = f"{config['experiment_name']}_{timestamp}"
    
    # 保存详细结果
    result_file = save_dir / f"{base_name}.json"
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    # 保存混淆矩阵
    cm_file = save_dir / f"{base_name}_confusion_matrix.png"
    plot_confusion_matrix(
        detailed_results['true_labels'],
        detailed_results['predictions'],
        cm_file
    )
    
    print(f"\nResults saved to: {save_dir}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_dir': '/home/z/Music/st/Kinetics-400/RQ1/FGSM_Image_Framewise',  # 视频文件目录
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',  # 标签文件路径
        'sample_size': 4000,  # 设置为None处理所有视频，或设置具体数字进行采样
        'random_seed': 42,
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ4/evaluationFramewise',  # 结果保存目录
        'experiment_name': 'timesformer_evaluation'  # 实验名称
    }
    
    results = evaluate_videos(config)

Using device: cuda
Loading model...


/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading labels...
Found 5081 video files
Sampled 4000 videos for evaluation


Processing videos:   0%|          | 0/4000 [00:00<?, ?it/s]/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  return torch.tensor(value)
Processing videos: 100%|██████████| 4000/4000 [10:11<00:00,  6.54it/s]
/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Processing completed with 0 errors

Performance Metrics:
Total samples processed: 4000
Accuracy: 0.4730
Precision: 0.5804
Recall: 0.4730
F1 Score: 0.4809
Average Confidence: 0.6284

Results saved to: /home/z/Music/st/Kinetics-400/RQ4/evaluationFramewise


Parser

In [2]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import glob
import seaborn as sns
import matplotlib.pyplot as plt

def load_video_frames(video_path, num_frames=8):
    """
    从单个视频文件中读取帧
    """
    frames = []
    try:
        container = av.open(video_path)
        for frame in container.decode(video=0):
            frames.append(frame.to_rgb().to_ndarray())
            if len(frames) >= num_frames:
                return np.stack(frames[:num_frames])
    except Exception as e:
        print(f"Error processing video {video_path}: {str(e)}")
        raise
    
    if len(frames) == 0:
        raise ValueError("No frames could be extracted")
    
    # 如果帧数不足，重复最后一帧
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames[:num_frames])

def load_labels(label_file):
    """
    加载标签文件
    """
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def plot_confusion_matrix(true_labels, predictions, save_path):
    """
    绘制并保存混淆矩阵
    """
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path)
    plt.close()

def evaluate_videos(config):
    # 设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 初始化模型
    print("Loading model...")
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    model.eval()  # 设置为评估模式
    
    # 加载标签
    print("Loading labels...")
    video_labels = load_labels(config['label_file'])
    
    # 准备结果存储
    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'confidence_scores': [],
        'model_name': config['model_name'],
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config
    }
    
    # 获取所有视频文件
    video_files = glob.glob(os.path.join(config['video_dir'], '*.mp4'))
    print(f"Found {len(video_files)} video files")
    
    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42))
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
            print(f"Sampled {config['sample_size']} videos for evaluation")
    
    # 处理每个视频
    error_count = 0
    for video_path in tqdm(video_files, desc="Processing videos"):
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # 跳过没有标签的视频
        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        
        try:
            # 读取视频帧并进行预测
            frames = load_video_frames(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probabilities = torch.nn.functional.softmax(logits, dim=-1)
                pred_label = logits.argmax(-1).cpu().numpy()[0]
                confidence = float(probabilities.max().cpu().numpy())
            
            # 存储结果
            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            detailed_results['confidence_scores'].append(confidence)
            
        except Exception as e:
            print(f"Error processing video {video_name}: {str(e)}")
            error_count += 1
            continue
    
    print(f"\nProcessing completed with {error_count} errors")
    
    # 计算评价指标
    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    
    # 添加汇总指标
    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions']),
        'error_count': error_count,
        'average_confidence': float(np.mean(detailed_results['confidence_scores']))
    }
    
    # 打印结果
    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Average Confidence: {detailed_results['summary']['average_confidence']:.4f}")
    
    # 保存结果
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = detailed_results['timestamp']
    base_name = f"{config['experiment_name']}_{timestamp}"
    
    # 保存详细结果
    result_file = save_dir / f"{base_name}.json"
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    # 保存混淆矩阵
    cm_file = save_dir / f"{base_name}_confusion_matrix.png"
    plot_confusion_matrix(
        detailed_results['true_labels'],
        detailed_results['predictions'],
        cm_file
    )
    
    print(f"\nResults saved to: {save_dir}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_dir': '/home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe',  # 视频文件目录
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',  # 标签文件路径
        'sample_size': 4000,  # 设置为None处理所有视频，或设置具体数字进行采样
        'random_seed': 42,
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ4/evaluationSparse',  # 结果保存目录
        'experiment_name': 'timesformer_evaluation2'  # 实验名称
    }
    
    results = evaluate_videos(config)

Using device: cuda
Loading model...


/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading labels...
Found 8075 video files
Sampled 4000 videos for evaluation


Processing videos:   1%|          | 38/4000 [00:06<14:49,  4.45it/s]Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found
Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found
Processing videos:   1%|          | 41/4000 [00:06<08:33,  7.71it/s]

Error processing video /home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/ZSYJC8QOoh8.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/ZSYJC8QOoh8.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found
Error processing video ZSYJC8QOoh8: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/ZSYJC8QOoh8.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found
Error processing video /home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/_lFH0L9EsmY.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/_lFH0L9EsmY.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found
Error processing video _lFH0L9EsmY: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/_lFH0L9EsmY.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] 

...

Error processing video /home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/xOOs_lJ-E2w.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/xOOs_lJ-E2w.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found
Error processing video xOOs_lJ-E2w: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/xOOs_lJ-E2w.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found


Processing videos: 100%|█████████▉| 3996/4000 [11:10<00:00,  6.11it/s]Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found
Processing videos: 100%|█████████▉| 3998/4000 [11:10<00:00,  7.89it/s]

Error processing video /home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/FJ6bs7-IKz0.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/FJ6bs7-IKz0.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found
Error processing video FJ6bs7-IKz0: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/Sparse_Keyframe/FJ6bs7-IKz0.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found


Processing videos: 100%|██████████| 4000/4000 [11:11<00:00,  5.96it/s]
/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Processing completed with 157 errors

Performance Metrics:
Total samples processed: 3843
Accuracy: 0.5308
Precision: 0.6058
Recall: 0.5308
F1 Score: 0.5320
Average Confidence: 0.6778

Results saved to: /home/z/Music/st/Kinetics-400/RQ4/evaluationSparse


r3d

In [3]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import glob
import seaborn as sns
import matplotlib.pyplot as plt

def load_video_frames(video_path, num_frames=8):
    """
    从单个视频文件中读取帧
    """
    frames = []
    try:
        container = av.open(video_path)
        for frame in container.decode(video=0):
            frames.append(frame.to_rgb().to_ndarray())
            if len(frames) >= num_frames:
                return np.stack(frames[:num_frames])
    except Exception as e:
        print(f"Error processing video {video_path}: {str(e)}")
        raise
    
    if len(frames) == 0:
        raise ValueError("No frames could be extracted")
    
    # 如果帧数不足，重复最后一帧
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames[:num_frames])

def load_labels(label_file):
    """
    加载标签文件
    """
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def plot_confusion_matrix(true_labels, predictions, save_path):
    """
    绘制并保存混淆矩阵
    """
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path)
    plt.close()

def evaluate_videos(config):
    # 设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 初始化模型
    print("Loading model...")
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    model.eval()  # 设置为评估模式
    
    # 加载标签
    print("Loading labels...")
    video_labels = load_labels(config['label_file'])
    
    # 准备结果存储
    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'confidence_scores': [],
        'model_name': config['model_name'],
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config
    }
    
    # 获取所有视频文件
    video_files = glob.glob(os.path.join(config['video_dir'], '*.mp4'))
    print(f"Found {len(video_files)} video files")
    
    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42))
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
            print(f"Sampled {config['sample_size']} videos for evaluation")
    
    # 处理每个视频
    error_count = 0
    for video_path in tqdm(video_files, desc="Processing videos"):
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # 跳过没有标签的视频
        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        
        try:
            # 读取视频帧并进行预测
            frames = load_video_frames(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probabilities = torch.nn.functional.softmax(logits, dim=-1)
                pred_label = logits.argmax(-1).cpu().numpy()[0]
                confidence = float(probabilities.max().cpu().numpy())
            
            # 存储结果
            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            detailed_results['confidence_scores'].append(confidence)
            
        except Exception as e:
            print(f"Error processing video {video_name}: {str(e)}")
            error_count += 1
            continue
    
    print(f"\nProcessing completed with {error_count} errors")
    
    # 计算评价指标
    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    
    # 添加汇总指标
    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions']),
        'error_count': error_count,
        'average_confidence': float(np.mean(detailed_results['confidence_scores']))
    }
    
    # 打印结果
    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Average Confidence: {detailed_results['summary']['average_confidence']:.4f}")
    
    # 保存结果
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = detailed_results['timestamp']
    base_name = f"{config['experiment_name']}_{timestamp}"
    
    # 保存详细结果
    result_file = save_dir / f"{base_name}.json"
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    # 保存混淆矩阵
    cm_file = save_dir / f"{base_name}_confusion_matrix.png"
    plot_confusion_matrix(
        detailed_results['true_labels'],
        detailed_results['predictions'],
        cm_file
    )
    
    print(f"\nResults saved to: {save_dir}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_dir': '/home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18',  # 视频文件目录
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',  # 标签文件路径
        'sample_size': 4000,  # 设置为None处理所有视频，或设置具体数字进行采样
        'random_seed': 42,
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ4/evaluationR3D',  # 结果保存目录
        'experiment_name': 'timesformer_evaluation3'  # 实验名称
    }
    
    results = evaluate_videos(config)

Using device: cuda
Loading model...


/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading labels...
Found 10266 video files
Sampled 4000 videos for evaluation


Processing videos:   0%|          | 3/4000 [00:00<12:05,  5.51it/s]Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found
Processing videos:   0%|          | 5/4000 [00:00<08:11,  8.13it/s]

Error processing video /home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/oi7OzVkguos.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/oi7OzVkguos.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found
Error processing video oi7OzVkguos: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/oi7OzVkguos.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found


...

Error processing video /home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/wjb2XoeM28M.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/wjb2XoeM28M.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found
Error processing video wjb2XoeM28M: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/wjb2XoeM28M.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found


Processing videos: 100%|█████████▉| 3993/4000 [11:15<00:01,  6.09it/s]Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found
Processing videos: 100%|█████████▉| 3995/4000 [11:15<00:00,  7.97it/s]

Error processing video /home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/gZtIGav0u9c.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/gZtIGav0u9c.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found
Error processing video gZtIGav0u9c: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/gZtIGav0u9c.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found


Processing videos: 100%|█████████▉| 3998/4000 [11:16<00:00,  7.00it/s]Format mov,mp4,m4a,3gp,3g2,mj2 detected only with low score of 1, misdetection possible!
moov atom not found
Processing videos: 100%|██████████| 4000/4000 [11:16<00:00,  5.91it/s]
/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Error processing video /home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/OyO-B-omyS0.mp4: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/OyO-B-omyS0.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found
Error processing video OyO-B-omyS0: [Errno 1094995529] Invalid data found when processing input: '/home/z/Music/st/Kinetics-400/RQ1/VBAD_r3d_18/OyO-B-omyS0.mp4'; last error log: [mov,mp4,m4a,3gp,3g2,mj2] moov atom not found

Processing completed with 102 errors

Performance Metrics:
Total samples processed: 3898
Accuracy: 0.1916
Precision: 0.3259
Recall: 0.1916
F1 Score: 0.1986
Average Confidence: 0.4212

Results saved to: /home/z/Music/st/Kinetics-400/RQ4/evaluationR3D


Original

In [4]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import glob
import seaborn as sns
import matplotlib.pyplot as plt

def load_video_frames(video_path, num_frames=8):
    """
    从单个视频文件中读取帧
    """
    frames = []
    try:
        container = av.open(video_path)
        for frame in container.decode(video=0):
            frames.append(frame.to_rgb().to_ndarray())
            if len(frames) >= num_frames:
                return np.stack(frames[:num_frames])
    except Exception as e:
        print(f"Error processing video {video_path}: {str(e)}")
        raise
    
    if len(frames) == 0:
        raise ValueError("No frames could be extracted")
    
    # 如果帧数不足，重复最后一帧
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames[:num_frames])

def load_labels(label_file):
    """
    加载标签文件
    """
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def plot_confusion_matrix(true_labels, predictions, save_path):
    """
    绘制并保存混淆矩阵
    """
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path)
    plt.close()

def evaluate_videos(config):
    # 设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 初始化模型
    print("Loading model...")
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    model.eval()  # 设置为评估模式
    
    # 加载标签
    print("Loading labels...")
    video_labels = load_labels(config['label_file'])
    
    # 准备结果存储
    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'confidence_scores': [],
        'model_name': config['model_name'],
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config
    }
    
    # 获取所有视频文件
    video_files = glob.glob(os.path.join(config['video_dir'], '*.mp4'))
    print(f"Found {len(video_files)} video files")
    
    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42))
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
            print(f"Sampled {config['sample_size']} videos for evaluation")
    
    # 处理每个视频
    error_count = 0
    for video_path in tqdm(video_files, desc="Processing videos"):
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # 跳过没有标签的视频
        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        
        try:
            # 读取视频帧并进行预测
            frames = load_video_frames(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probabilities = torch.nn.functional.softmax(logits, dim=-1)
                pred_label = logits.argmax(-1).cpu().numpy()[0]
                confidence = float(probabilities.max().cpu().numpy())
            
            # 存储结果
            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            detailed_results['confidence_scores'].append(confidence)
            
        except Exception as e:
            print(f"Error processing video {video_name}: {str(e)}")
            error_count += 1
            continue
    
    print(f"\nProcessing completed with {error_count} errors")
    
    # 计算评价指标
    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    
    # 添加汇总指标
    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions']),
        'error_count': error_count,
        'average_confidence': float(np.mean(detailed_results['confidence_scores']))
    }
    
    # 打印结果
    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Average Confidence: {detailed_results['summary']['average_confidence']:.4f}")
    
    # 保存结果
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = detailed_results['timestamp']
    base_name = f"{config['experiment_name']}_{timestamp}"
    
    # 保存详细结果
    result_file = save_dir / f"{base_name}.json"
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    # 保存混淆矩阵
    cm_file = save_dir / f"{base_name}_confusion_matrix.png"
    plot_confusion_matrix(
        detailed_results['true_labels'],
        detailed_results['predictions'],
        cm_file
    )
    
    print(f"\nResults saved to: {save_dir}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_dir': '/home/z/Music/st/Kinetics-400/RQ1/videos_val',  # 视频文件目录
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',  # 标签文件路径
        'sample_size': 5000,  # 设置为None处理所有视频，或设置具体数字进行采样
        'random_seed': 42,
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ4/evaluationOrriginal',  # 结果保存目录
        'experiment_name': 'timesformer_evaluation4'  # 实验名称
    }
    
    results = evaluate_videos(config)

Using device: cuda
Loading model...


/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading labels...
Found 19796 video files
Sampled 5000 videos for evaluation


Processing videos: 100%|██████████| 5000/5000 [14:25<00:00,  5.78it/s]
/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Processing completed with 0 errors

Performance Metrics:
Total samples processed: 5000
Accuracy: 0.5444
Precision: 0.6100
Recall: 0.5444
F1 Score: 0.5457
Average Confidence: 0.6781

Results saved to: /home/z/Music/st/Kinetics-400/RQ4/evaluationOrriginal


My method

In [5]:
import torch
import av
import numpy as np
from tqdm import tqdm
import os
import json
from datetime import datetime
import pathlib
from transformers import TimesformerForVideoClassification, AutoImageProcessor
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix
import glob
import seaborn as sns
import matplotlib.pyplot as plt

def load_video_frames(video_path, num_frames=8):
    """
    从单个视频文件中读取帧
    """
    frames = []
    try:
        container = av.open(video_path)
        for frame in container.decode(video=0):
            frames.append(frame.to_rgb().to_ndarray())
            if len(frames) >= num_frames:
                return np.stack(frames[:num_frames])
    except Exception as e:
        print(f"Error processing video {video_path}: {str(e)}")
        raise
    
    if len(frames) == 0:
        raise ValueError("No frames could be extracted")
    
    # 如果帧数不足，重复最后一帧
    while len(frames) < num_frames:
        frames.append(frames[-1])
    
    return np.stack(frames[:num_frames])

def load_labels(label_file):
    """
    加载标签文件
    """
    labels = {}
    with open(label_file, 'r') as f:
        for line in f:
            video_name, label = line.strip().split()
            labels[video_name.split('.')[0]] = int(label)
    return labels

def plot_confusion_matrix(true_labels, predictions, save_path):
    """
    绘制并保存混淆矩阵
    """
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(save_path)
    plt.close()

def evaluate_videos(config):
    # 设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # 初始化模型
    print("Loading model...")
    model = TimesformerForVideoClassification.from_pretrained(config['model_name']).to(device)
    processor = AutoImageProcessor.from_pretrained(config['model_name'])
    model.eval()  # 设置为评估模式
    
    # 加载标签
    print("Loading labels...")
    video_labels = load_labels(config['label_file'])
    
    # 准备结果存储
    detailed_results = {
        'predictions': [],
        'true_labels': [],
        'video_names': [],
        'confidence_scores': [],
        'model_name': config['model_name'],
        'timestamp': datetime.now().strftime("%Y%m%d_%H%M%S"),
        'config': config
    }
    
    # 获取所有视频文件
    video_files = glob.glob(os.path.join(config['video_dir'], '*.mp4'))
    print(f"Found {len(video_files)} video files")
    
    if config.get('sample_size'):
        if len(video_files) > config['sample_size']:
            np.random.seed(config.get('random_seed', 42))
            video_files = np.random.choice(video_files, config['sample_size'], replace=False)
            print(f"Sampled {config['sample_size']} videos for evaluation")
    
    # 处理每个视频
    error_count = 0
    for video_path in tqdm(video_files, desc="Processing videos"):
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        
        # 跳过没有标签的视频
        if video_name not in video_labels:
            continue
            
        true_label = video_labels[video_name]
        
        try:
            # 读取视频帧并进行预测
            frames = load_video_frames(video_path)
            inputs = processor(list(frames), return_tensors="pt").to(device)
            
            with torch.no_grad():
                outputs = model(**inputs)
                logits = outputs.logits
                probabilities = torch.nn.functional.softmax(logits, dim=-1)
                pred_label = logits.argmax(-1).cpu().numpy()[0]
                confidence = float(probabilities.max().cpu().numpy())
            
            # 存储结果
            detailed_results['predictions'].append(int(pred_label))
            detailed_results['true_labels'].append(int(true_label))
            detailed_results['video_names'].append(video_name)
            detailed_results['confidence_scores'].append(confidence)
            
        except Exception as e:
            print(f"Error processing video {video_name}: {str(e)}")
            error_count += 1
            continue
    
    print(f"\nProcessing completed with {error_count} errors")
    
    # 计算评价指标
    precision, recall, f1, _ = precision_recall_fscore_support(
        detailed_results['true_labels'], 
        detailed_results['predictions'], 
        average='weighted'
    )
    accuracy = accuracy_score(
        detailed_results['true_labels'], 
        detailed_results['predictions']
    )
    
    # 添加汇总指标
    detailed_results['summary'] = {
        'accuracy': float(accuracy),
        'precision': float(precision),
        'recall': float(recall),
        'f1': float(f1),
        'total_samples': len(detailed_results['predictions']),
        'error_count': error_count,
        'average_confidence': float(np.mean(detailed_results['confidence_scores']))
    }
    
    # 打印结果
    print("\nPerformance Metrics:")
    print(f"Total samples processed: {detailed_results['summary']['total_samples']}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Average Confidence: {detailed_results['summary']['average_confidence']:.4f}")
    
    # 保存结果
    save_dir = pathlib.Path(config['results_directory'])
    save_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = detailed_results['timestamp']
    base_name = f"{config['experiment_name']}_{timestamp}"
    
    # 保存详细结果
    result_file = save_dir / f"{base_name}.json"
    with open(result_file, 'w') as f:
        json.dump(detailed_results, f, indent=4)
    
    # 保存混淆矩阵
    cm_file = save_dir / f"{base_name}_confusion_matrix.png"
    plot_confusion_matrix(
        detailed_results['true_labels'],
        detailed_results['predictions'],
        cm_file
    )
    
    print(f"\nResults saved to: {save_dir}")
    
    return detailed_results

if __name__ == "__main__":
    config = {
        'model_name': 'facebook/timesformer-base-finetuned-k400',
        'video_dir': '/home/z/Music/st/Kinetics-400/RQ1/New_merged_FGSM',  # 视频文件目录
        'label_file': '/home/z/Music/st/Kinetics-400/RQ1/kinetics400_val_list_videos.txt',  # 标签文件路径
        'sample_size': 5000,  # 设置为None处理所有视频，或设置具体数字进行采样
        'random_seed': 42,
        'results_directory': '/home/z/Music/st/Kinetics-400/RQ4/evaluationOrriginal',  # 结果保存目录
        'experiment_name': 'timesformer_evaluation4'  # 实验名称
    }
    
    results = evaluate_videos(config)

Using device: cuda
Loading model...


/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading labels...
Found 19796 video files
Sampled 5000 videos for evaluation


Processing videos: 100%|██████████| 5000/5000 [13:13<00:00,  6.30it/s]
/home/z/miniconda3/envs/mmxai/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Processing completed with 0 errors

Performance Metrics:
Total samples processed: 5000
Accuracy: 0.1388
Precision: 0.2351
Recall: 0.1388
F1 Score: 0.1362
Average Confidence: 0.4514

Results saved to: /home/z/Music/st/Kinetics-400/RQ4/evaluationOrriginal
